## Optimizadores, Learning Rate Schedulers

En esta sesión vamos a comparar el resultado diferentes optimizadores y learning rates schedulers 

## Leer, normalizar y particionar datos

In [1]:
## Importar y normalizar datos

from tensorflow import keras
from keras.datasets import mnist

(x_train1, y_train1), (x_test, y_test) = mnist.load_data()

print('training set', x_train1.shape)
print('test set', x_test.shape)

x_train1 = x_train1.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train1 = x_train1.astype('float32')
x_test = x_test.astype('float32')

# Normalize [0..255]-->[0..1]
x_train1 /= 255
x_test /= 255

# convert class vectors to binary class matrices
num_classes=10
y_train1 = keras.utils.to_categorical(y_train1, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train1, y_train1, test_size=0.2, random_state=42)

print('training set', x_train.shape)
print('val set', x_val.shape)

2023-10-25 15:02:30.212314: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-25 15:02:30.254313: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-25 15:02:30.450709: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-25 15:02:30.450736: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-25 15:02:30.451873: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

training set (60000, 28, 28)
test set (10000, 28, 28)
training set (48000, 784)
val set (12000, 784)


## Optimizadores

Vamos a probar diferentes optimizadores:

In [2]:
from keras import Sequential
from keras.layers import Dense, Input
from keras.optimizers import SGD,Adam,Adagrad

batch_size=128
epochs=5
lr=0.001

opt=[]
opt.append(SGD(learning_rate=lr, momentum=0.9))
opt.append(Adam(learning_rate=lr))
opt.append(Adagrad(learning_rate=lr))

best_acc=0.0
for optim in opt:
    model = Sequential()

    model.add(Input(784))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
              optimizer=optim,
              metrics=['accuracy'])
    
 

    history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val)) 
    
    print("\n>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>\n")
    
    if history.history['val_accuracy'][-1]>best_acc:
        best_acc=history.history['val_accuracy'][-1]
        bestopt=optim

print("=============================")
print("Best acc",best_acc)
print("Best optim",bestopt)
print("=============================")


Epoch 1/5
375/375 [==============================] - 3s 8ms/step - loss: 1.2036 - accuracy: 0.7403 - val_loss: 0.6068 - val_accuracy: 0.8558
Epoch 2/5
375/375 [==============================] - 3s 8ms/step - loss: 0.4885 - accuracy: 0.8780 - val_loss: 0.4130 - val_accuracy: 0.8896
Epoch 3/5
375/375 [==============================] - 3s 8ms/step - loss: 0.3801 - accuracy: 0.8973 - val_loss: 0.3513 - val_accuracy: 0.9025
Epoch 4/5
375/375 [==============================] - 4s 10ms/step - loss: 0.3341 - accuracy: 0.9074 - val_loss: 0.3153 - val_accuracy: 0.9120
Epoch 5/5
375/375 [==============================] - 3s 9ms/step - loss: 0.3058 - accuracy: 0.9145 - val_loss: 0.2936 - val_accuracy: 0.9175

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

Epoch 1/5
375/375 [==============================] - 5s 11ms/step - loss: 0.2141 - accuracy: 0.9355 - val_loss: 0.1111 - val_accuracy: 0.9652
Epoch 2/5
375/375 [==============================] - 4s 12ms/step - loss: 0.0806 - accuracy

## **EJERCICIO**

Añade más epochs a este ejemplo anterior, un early stopping y un model_checkpoint. Luego prueba el test sobre el model checkpoint almacenado.

In [2]:
from keras import Sequential
from keras.layers import Dense, Input
from keras.optimizers import SGD,Adam,Adagrad

batch_size=128
epochs=100
lr=0.001

opt=[]
opt.append(SGD(learning_rate=lr, momentum=0.9))
opt.append(Adam(learning_rate=lr))
opt.append(Adagrad(learning_rate=lr))

best_acc=0.0
for optim in opt:
    model = Sequential()

    model.add(Input(784))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
              optimizer=optim,
              metrics=['accuracy'])

    callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3)
    checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

    history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[callback, checkpoint]) 
    
    print("\n>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>\n")
    
    if history.history['val_accuracy'][-1]>best_acc:
        best_acc=history.history['val_accuracy'][-1]
        bestopt=optim

print("=============================")
print("Best acc",best_acc)
print("Best optim",bestopt)
print("=============================")


## cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/100
371/375 [============================>.] - ETA: 0s - loss: 1.2512 - accuracy: 0.7348
Epoch 1: val_accuracy improved from -inf to 0.85817, saving model to best_model.h5
375/375 [==============================] - 3s 8ms/step - loss: 1.2444 - accuracy: 0.7360 - val_loss: 0.6137 - val_accuracy: 0.8582
Epoch 2/100
 24/375 [>.............................] - ETA: 2s - loss: 0.6072 - accuracy: 0.8587

/home/pegi/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


369/375 [============================>.] - ETA: 0s - loss: 0.4908 - accuracy: 0.8762
Epoch 2: val_accuracy improved from 0.85817 to 0.89058, saving model to best_model.h5
375/375 [==============================] - 3s 8ms/step - loss: 0.4890 - accuracy: 0.8765 - val_loss: 0.4140 - val_accuracy: 0.8906
Epoch 3/100
372/375 [============================>.] - ETA: 0s - loss: 0.3793 - accuracy: 0.8969
Epoch 3: val_accuracy improved from 0.89058 to 0.90458, saving model to best_model.h5
375/375 [==============================] - 3s 8ms/step - loss: 0.3789 - accuracy: 0.8971 - val_loss: 0.3497 - val_accuracy: 0.9046
Epoch 4/100
371/375 [============================>.] - ETA: 0s - loss: 0.3331 - accuracy: 0.9072
Epoch 4: val_accuracy improved from 0.90458 to 0.91200, saving model to best_model.h5
375/375 [==============================] - 3s 9ms/step - loss: 0.3331 - accuracy: 0.9071 - val_loss: 0.3157 - val_accuracy: 0.9120
Epoch 5/100
371/375 [============================>.] - ETA: 0s - loss:

## Learning Rates Schedulers

Los learning rate schedulers son mecanismos de modificación del learning rate. Normalmente lo que hacen es reducir el valor del learning rate, lo que se conoce como ***learning rate annealing***. Esta modificación se suele realizar al acabar cada epoch.

Keras ya dispone de algunos learning rate schedulers implementados pero el usuario puede implemetar su propia estrategia de annealing. Veamos ambos casos.

### LRS ya implementado en Keras

In [5]:
# Emplear un LRS estandard de Keras: ReduceLROnPlateau
epochs = 5
from keras.callbacks import ReduceLROnPlateau

model = Sequential()

model.add(Input(784))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

opt=Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                patience=2, min_lr=0.00001)

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[reduce_lr])  ## <--- aquí está



Epoch 1/5
375/375 [==============================] - 5s 13ms/step - loss: 0.3721 - accuracy: 0.9089 - val_loss: 0.2271 - val_accuracy: 0.9348 - lr: 0.0100
Epoch 2/5
375/375 [==============================] - 4s 12ms/step - loss: 0.1643 - accuracy: 0.9516 - val_loss: 0.1491 - val_accuracy: 0.9587 - lr: 0.0100
Epoch 3/5
375/375 [==============================] - 5s 12ms/step - loss: 0.1295 - accuracy: 0.9632 - val_loss: 0.1477 - val_accuracy: 0.9577 - lr: 0.0100
Epoch 4/5
375/375 [==============================] - 5s 14ms/step - loss: 0.1127 - accuracy: 0.9683 - val_loss: 0.1808 - val_accuracy: 0.9563 - lr: 0.0100
Epoch 5/5
375/375 [==============================] - 5s 13ms/step - loss: 0.1063 - accuracy: 0.9700 - val_loss: 0.1621 - val_accuracy: 0.9596 - lr: 0.0100


### LRS propio mediante función 

La función toma como entrada el epoch

In [6]:
## Emplear un LRS propio: LearningRateScheduler

from keras.callbacks import LearningRateScheduler

def scheduler(epoch):
    if epoch < 5:
        return lr
    else:
        return lr * 0.1

LRS=LearningRateScheduler(scheduler, verbose=1)

model = Sequential()

model.add(Input(784))

model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

opt=Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])   

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[LRS])
                



Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/5
375/375 [==============================] - 5s 13ms/step - loss: 0.2116 - accuracy: 0.9365 - val_loss: 0.0971 - val_accuracy: 0.9716 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/5
375/375 [==============================] - 5s 13ms/step - loss: 0.0800 - accuracy: 0.9751 - val_loss: 0.0932 - val_accuracy: 0.9718 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/5
375/375 [==============================] - 5s 13ms/step - loss: 0.0480 - accuracy: 0.9844 - val_loss: 0.0779 - val_accuracy: 0.9777 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.001.
Epoch 4/5
375/375 [==============================] - 5s 13ms/step - loss: 0.0346 - accuracy: 0.9890 - val_loss: 0.0852 - val_accuracy: 0.9777 - lr: 0.0010

Epoch 5: LearningRateScheduler setting learning rate to 0.001.
Epoch 5/5
375/375 [==============================] - 5s 13ms

### LRS propio tipo CosineAnnealing

![cosine annealing](cosine.png)


In [7]:
# Emplear un LRS propio, CosineAnnealingScheduler

from keras.callbacks import LearningRateScheduler
import math

lr_max=0.001
lr_min=0.00001
epochs=100
def cosine_annealing(x): # recordemos, x es el número de epoch
    lr = lr_max/2 * (1 + math.cos(math.pi * x / epochs))
    if lr<lr_min:
        lr=lr_min
    return lr

LRS = LearningRateScheduler(cosine_annealing)

model = Sequential()

model.add(Input(784))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

opt=Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[LRS])



Epoch 1/100
375/375 [==============================] - 5s 13ms/step - loss: 0.2125 - accuracy: 0.9356 - val_loss: 0.1679 - val_accuracy: 0.9468 - lr: 0.0010
Epoch 2/100
375/375 [==============================] - 5s 13ms/step - loss: 0.0784 - accuracy: 0.9755 - val_loss: 0.1005 - val_accuracy: 0.9697 - lr: 9.9975e-04
Epoch 3/100
375/375 [==============================] - 5s 13ms/step - loss: 0.0537 - accuracy: 0.9826 - val_loss: 0.0935 - val_accuracy: 0.9718 - lr: 9.9901e-04
Epoch 4/100
375/375 [==============================] - 5s 13ms/step - loss: 0.0354 - accuracy: 0.9884 - val_loss: 0.0918 - val_accuracy: 0.9736 - lr: 9.9778e-04
Epoch 5/100
375/375 [==============================] - 5s 13ms/step - loss: 0.0262 - accuracy: 0.9916 - val_loss: 0.0875 - val_accuracy: 0.9776 - lr: 9.9606e-04
Epoch 6/100
375/375 [==============================] - 5s 13ms/step - loss: 0.0239 - accuracy: 0.9919 - val_loss: 0.0819 - val_accuracy: 0.9787 - lr: 9.9384e-04
Epoch 7/100
375/375 [=================

### LRS propio tipo CosineAnnealing with restarts

![cosine annealing restarts](cosinerestart.png)

Este scheduler tiene sentido cuanto queremos guardarnos cada uno de los estado del modelo alcanzado en el mínimo LR para luego combinarlos.

In [8]:
# Emplear un LRS propio, CosineAnnealingScheduler with restarts

from keras.callbacks import LearningRateScheduler
import math

lr_max=0.001
lr_min=0.00001
epochs=100

def cosine_annealing_with_restarts(x):
    lr = lr_max/2 * (1 + math.cos(math.pi * (x % (epochs/5)) / (epochs/5)))
    if lr<lr_min:
        lr=lr_min
    return lr

LRS = LearningRateScheduler(cosine_annealing_with_restarts)

model = Sequential()
model.add(Input(784))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

opt=Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[LRS])

Epoch 1/100
375/375 [==============================] - 5s 13ms/step - loss: 0.2127 - accuracy: 0.9363 - val_loss: 0.1070 - val_accuracy: 0.9672 - lr: 0.0010
Epoch 2/100
375/375 [==============================] - 5s 13ms/step - loss: 0.0785 - accuracy: 0.9751 - val_loss: 0.0861 - val_accuracy: 0.9743 - lr: 9.9384e-04
Epoch 3/100
375/375 [==============================] - 5s 13ms/step - loss: 0.0505 - accuracy: 0.9838 - val_loss: 0.0910 - val_accuracy: 0.9728 - lr: 9.7553e-04
Epoch 4/100
375/375 [==============================] - 5s 13ms/step - loss: 0.0354 - accuracy: 0.9881 - val_loss: 0.0827 - val_accuracy: 0.9768 - lr: 9.4550e-04
Epoch 5/100
375/375 [==============================] - 5s 13ms/step - loss: 0.0236 - accuracy: 0.9921 - val_loss: 0.0783 - val_accuracy: 0.9800 - lr: 9.0451e-04
Epoch 6/100
375/375 [==============================] - 5s 12ms/step - loss: 0.0207 - accuracy: 0.9930 - val_loss: 0.1053 - val_accuracy: 0.9753 - lr: 8.5355e-04
Epoch 7/100
375/375 [=================

## Ejercicio

Prueba los diferentes LRS sobre el modelo anterior. Emplea la que creas que es la mejor topología de red y optimizador. Emplea early stopping y model checkpoint. Calcula el accuracy sobre el test con el modelo almacenado en el model checkpoint.



In [12]:
## Solución
from keras import Sequential
from keras.layers import Dense, Input
from keras.optimizers import SGD,Adam,Adagrad

batch_size=128
epochs=100
lr=0.01

opt_adam = Adam(learning_rate=lr)

best_acc=0.0

model = Sequential()

model.add(Input(784))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
            optimizer=opt_adam,
            metrics=['accuracy'])

callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3)
checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
LRS = LearningRateScheduler(cosine_annealing)


history = model.fit(x_train, y_train,
                batch_size=batch_size,
                epochs=epochs,
                verbose=1,
                validation_data=(x_val, y_val),
                callbacks=[callback, checkpoint, LRS]) 


print("=============================")
print("best_acc: ", history.history['val_accuracy'][-1])
print("=============================")


## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])



Epoch 1/100
374/375 [============================>.] - ETA: 0s - loss: 0.2126 - accuracy: 0.9355
Epoch 1: val_accuracy improved from -inf to 0.96592, saving model to best_model.h5
375/375 [==============================] - 4s 11ms/step - loss: 0.2122 - accuracy: 0.9356 - val_loss: 0.1079 - val_accuracy: 0.9659 - lr: 0.0010
Epoch 2/100
370/375 [============================>.] - ETA: 0s - loss: 0.0787 - accuracy: 0.9754
Epoch 2: val_accuracy improved from 0.96592 to 0.97408, saving model to best_model.h5
375/375 [==============================] - 4s 10ms/step - loss: 0.0789 - accuracy: 0.9754 - val_loss: 0.0830 - val_accuracy: 0.9741 - lr: 9.9975e-04
Epoch 3/100
373/375 [============================>.] - ETA: 0s - loss: 0.0488 - accuracy: 0.9846
Epoch 3: val_accuracy did not improve from 0.97408
375/375 [==============================] - 4s 10ms/step - loss: 0.0487 - accuracy: 0.9846 - val_loss: 0.0917 - val_accuracy: 0.9722 - lr: 9.9901e-04
Epoch 4/100
375/375 [=========================

---

## EXTRA, estandaricemos la clase MLP

Vamos a implementar una clase para ahorrarnos escribir código recurrentemente. 
Para ello empleamos la **functional api** de Keras:

In [ ]:
from tensorflow import keras
from keras.layers import Dense

class MLP(keras.Model):

  def __init__(self, input_size,num_classes,hidden=[128]):
    super().__init__()
    
    self.hidden = []
    self.num_h=len(hidden)

    for h in hidden:
       self.hidden.append(Dense(h, activation='relu'))
    self.out = Dense(num_classes, activation='softmax')
    

  def call(self, inputs, training=False):
    
    x=self.hidden[0](inputs)
    for h in range(1,self.num_h):
        x = self.hidden[h](x)
    x = self.out(x)
    return x
    


model = MLP(784,10,[1024,512])  ## <-- aquí se instancia el modelo, input, num clases y lista con hidden layers
model.build((None,784)) ## Esto es necesario para poder instaciar adecuadamente todos los shapes del grafo de computación


model.summary()